In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Caricamento del dataframe
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")

In [3]:
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################
import pandas as pd
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
# aug = CSVUtils.get_df_from_csv("./augumentation.csv", "./newAugmentationDataset/")
# df_train = pd.concat([df_train, aug], ignore_index=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################

from torchvision import transforms
import torch

transform_func = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(2, 9),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train = AgeDatasetKL(df_train, path_col="path", label_col="age", label_function="Linear", 
                        transform_func=transform_func)
cd_train.set_n_classes(81)
cd_train.set_starting_class(1)
dm_train = CustomDataLoader(cd_train)
dl_train = dm_train.get_unbalanced_dataloader(batch_size=64, shuffle=True, drop_last=True, num_workers=8, prefetch_factor=4, pin_memory=True)
dl_train_balanced = dm_train.get_balanced_class_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                           batch_size=64, num_workers=8, prefetch_factor=4, pin_memory=True)
print(len(dl_train), len(dl_train_balanced))
cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)
cd_val.set_n_classes(81)
cd_val.set_starting_class(1)
validator = Validator(cd_val, AgeConversion.EVAge, 32, num_workers=8, prefetch_factor=4)

6739 6739


In [4]:
from ResNetFilmed.resnet import ResNetFiLMed, BackBone, ResNetNotFiLMed, DoNothingLayer
from torchvision.models import resnet18, ResNet18_Weights, efficientnet_b0, EfficientNet_B0_Weights
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn

####################################################
EPOCHS = 12
####################################################

backbone = resnet18(ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = DoNothingLayer()
backbone.train()
backbone.requires_grad_(True)
backbone.to("cuda")
model_age = ResNetNotFiLMed(backbone, 81)
model_age.load_state_dict(torch.load("./model_age_classification_simple.pt", map_location="cuda:0"))
opt = optim.SGD(set([*backbone.parameters(), *model_age.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)
kl = nn.KLDivLoss(reduction="batchmean")

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [5]:
def forward_function(x):
    out = model_age(x)
    out = F.softmax(out, dim=-1)
    return out

mae, val_aar, val_aar_old = validator.validate_ext(forward_function)
print(mae, val_aar, val_aar_old)

100%|██████████| 4493/4493 [01:51<00:00, 40.16 batch/s]


tensor(2.4615, dtype=torch.float64) tensor(6.2916, dtype=torch.float64) tensor(6.5774, dtype=torch.float64)


In [6]:
best_val_aar = -1

for e in range(EPOCHS):
    with tqdm(zip(dl_train, dl_train_balanced), unit=" batch") as tepoch:
        for batch, batch_balanced in tepoch:
            opt.zero_grad()
            x, y = batch
            x_bal, y_bal = batch_balanced

            x = x.to("cuda")
            y_age: torch.Tensor = y[0].to("cuda")
            y_age_kl: torch.Tensor = y[1].to("cuda")

            x_bal = x_bal.to("cuda")
            y_age_bal: torch.Tensor = y_bal[0].to("cuda")
            y_age_kl_bal: torch.Tensor = y_bal[1].to("cuda")

            out_age = model_age(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)
            out_rep = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out_rep).to("cuda")
            loss_age = torch.mean(torch.abs(y_age - out))
            loss = loss_age_kl + loss_age

            out_age_bal = model_age(x_bal)
            loss_age_kl_bal: torch.Tensor = kl(F.log_softmax(out_age_bal, dim=-1), y_age_kl_bal)
            out_bal_rep = F.softmax(out_age_bal, dim=-1)
            out_bal = AgeConversion.EVAge(out_bal_rep).to("cuda")
            loss_age_bal = torch.mean(torch.abs(y_age_bal - out_bal))
            loss_bal = loss_age_kl_bal + loss_age_bal

            # repr_mean = ((out_rep + out_bal_rep) / 2).detach()
            # loss_repr = torch.mean(torch.square(out_rep - repr_mean))
            # loss_repr_bal = torch.mean(torch.square(out_bal_rep - repr_mean))

            total_loss = loss + loss_bal#  + loss_repr + loss_repr_bal

            total_loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy(), loss_age=loss_age.detach().cpu().numpy(),
                                loss_age_kl_bal=loss_age_kl_bal.detach().cpu().numpy(), loss_age_bal=loss_age_bal.detach().cpu().numpy(),
                                # loss_repr=loss_repr.detach().cpu().numpy(), loss_repr_bal=loss_repr_bal.detach().cpu().numpy(),
                                total_loss=total_loss.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out

    ae, mae, val_aar, val_aar_old = validator.validate_ext2(forward_function)
    print(ae, mae, val_aar, val_aar_old)

    if best_val_aar < val_aar:
        best_val_aar = val_aar
        torch.save(model_age.state_dict(), "./model_age_feature_simple.pt")
        print("Saved model")

6739 batch [15:17,  7.35 batch/s, loss_age=3.2953024, loss_age_bal=2.9983397, loss_age_kl=1.4952563875675526, loss_age_kl_bal=1.4769178541097103, total_loss=9.265816285744645] 
100%|██████████| 4493/4493 [01:47<00:00, 41.90 batch/s]


{0: tensor(12.4916, dtype=torch.float64), 1: tensor(4.0337, dtype=torch.float64), 2: tensor(2.4652, dtype=torch.float64), 3: tensor(3.2688, dtype=torch.float64), 4: tensor(3.9627, dtype=torch.float64), 5: tensor(3.5057, dtype=torch.float64), 6: tensor(5.3551, dtype=torch.float64), 7: tensor(8.4105, dtype=torch.float64)} tensor(3.4465, dtype=torch.float64) tensor(1.2632, dtype=torch.float64) tensor(3.5535, dtype=torch.float64)
Saved model


6739 batch [15:21,  7.31 batch/s, loss_age=3.694905, loss_age_bal=3.448769, loss_age_kl=1.5013243234789924, loss_age_kl_bal=1.4173864740318298, total_loss=10.062384932718953]  
100%|██████████| 4493/4493 [01:48<00:00, 41.54 batch/s]


{0: tensor(10.5539, dtype=torch.float64), 1: tensor(4.5851, dtype=torch.float64), 2: tensor(2.8510, dtype=torch.float64), 3: tensor(3.4355, dtype=torch.float64), 4: tensor(3.9388, dtype=torch.float64), 5: tensor(3.9683, dtype=torch.float64), 6: tensor(4.2800, dtype=torch.float64), 7: tensor(7.2338, dtype=torch.float64)} tensor(3.6188, dtype=torch.float64) tensor(2.1853, dtype=torch.float64) tensor(3.5664, dtype=torch.float64)
Saved model


6739 batch [14:17,  7.86 batch/s, loss_age=4.283989, loss_age_bal=3.1754408, loss_age_kl=1.7574919478749933, loss_age_kl_bal=1.3957879471227277, total_loss=10.612709635903482] 
100%|██████████| 4493/4493 [01:47<00:00, 41.82 batch/s]


...

6739 batch [13:54,  8.08 batch/s, loss_age=3.4725785, loss_age_bal=3.6331632, loss_age_kl=1.457805011384024, loss_age_kl_bal=1.5296597851320186, total_loss=10.093206535789115] 
100%|██████████| 4493/4493 [01:48<00:00, 41.39 batch/s]


{0: tensor(9.0794, dtype=torch.float64), 1: tensor(3.8506, dtype=torch.float64), 2: tensor(2.6318, dtype=torch.float64), 3: tensor(3.3492, dtype=torch.float64), 4: tensor(4.6736, dtype=torch.float64), 5: tensor(4.5973, dtype=torch.float64), 6: tensor(4.7433, dtype=torch.float64), 7: tensor(6.2736, dtype=torch.float64)} tensor(3.7776, dtype=torch.float64) tensor(2.9136, dtype=torch.float64) tensor(4.0358, dtype=torch.float64)


6739 batch [14:48,  7.59 batch/s, loss_age=3.6234336, loss_age_bal=3.182855, loss_age_kl=1.4865563828598152, loss_age_kl_bal=1.2497961954538868, total_loss=9.54264105907237]   
100%|██████████| 4493/4493 [01:48<00:00, 41.36 batch/s]


{0: tensor(7.9253, dtype=torch.float64), 1: tensor(3.3165, dtype=torch.float64), 2: tensor(2.6181, dtype=torch.float64), 3: tensor(3.4179, dtype=torch.float64), 4: tensor(3.4873, dtype=torch.float64), 5: tensor(3.2767, dtype=torch.float64), 6: tensor(4.0922, dtype=torch.float64), 7: tensor(6.1591, dtype=torch.float64)} tensor(3.2608, dtype=torch.float64) tensor(3.7341, dtype=torch.float64) tensor(4.7599, dtype=torch.float64)


286 batch [00:40,  6.69 batch/s, loss_age=2.843449, loss_age_bal=3.5708604, loss_age_kl=1.34516212236695, loss_age_kl_bal=1.4708502811209296, total_loss=9.23032190513583]     

In [ ]:
####################################################
EPOCHS = 12
####################################################

dl_train = dm_train.get_balanced_class_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                  batch_size=64, num_workers=12, prefetch_factor=4, pin_memory=True)

model_age.load_state_dict(torch.load("./model_age_feature_simple.pt"))
opt = optim.SGD(set([*model_age.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)

In [ ]:
def forward_function(x):
    out = model_age(x)
    out = F.softmax(out, dim=-1)
    return out

mae, val_aar, val_aar_old = validator.validate_ext(forward_function)
print(mae, val_aar, val_aar_old)

100%|██████████| 4493/4493 [02:21<00:00, 31.72 batch/s]


tensor(2.2409, dtype=torch.float64) tensor(4.9902, dtype=torch.float64) tensor(5.8763, dtype=torch.float64)


In [ ]:
best_val_aar = val_aar

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age: torch.Tensor = y[0].to("cuda")
            y_age_kl: torch.Tensor = y[1].to("cuda")

            out_age = model_age(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age = torch.mean(torch.abs(y_age - out))

            loss = loss_age_kl + torch.square(loss_age - mae)
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy(), loss_age=loss_age.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out

    val_aar, val_aar_old = validator.validate(forward_function)
    print(val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age.state_dict(), "./model_age_classification_simple.pt")
        print("Saved model")

100%|██████████| 4493/4493 [01:58<00:00, 38.06 batch/s]


tensor(4.3330, dtype=torch.float64) tensor(3.5900, dtype=torch.float64)


100%|██████████| 4493/4493 [02:03<00:00, 36.30 batch/s]


tensor(3.4237, dtype=torch.float64) tensor(2.7052, dtype=torch.float64)


100%|██████████| 4493/4493 [02:19<00:00, 32.31 batch/s]


...

100%|██████████| 4493/4493 [01:53<00:00, 39.44 batch/s]


tensor(5.2628, dtype=torch.float64) tensor(4.7096, dtype=torch.float64)


100%|██████████| 4493/4493 [01:51<00:00, 40.46 batch/s]


tensor(5.3480, dtype=torch.float64) tensor(4.7429, dtype=torch.float64)


: 